# Спринт 17 «Машинное обучение для текстов»

# Проект для «Викишоп»

## Навигация

1. [Описание проекта](#Описание-проекта)
1. [Описание данных](#Описание-данных)
1. [Предобработка данных](#Предобработка-данных)
1. [Обучение](#Обучение)
   - [LogReg](#LogReg)
   - [CatBoost](#CatBoost)
1. [Общие выводы](#Общие-выводы)

## Описание проекта

[к навигации](#Навигация)

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классификации комментариев на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

## Описание данных

[к навигации](#Навигация)

Столбец `text` содержит текст комментария, а `toxic` — целевой признак.

In [1]:
import os
import re

import catboost
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV

import spacy
from tqdm import tqdm
tqdm.pandas()

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mikhail\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
DATA_PATH = os.path.join('..', 'datasets', 'toxic_comments.csv')

RANDOM_STATE = 42

## Предобработка данных

[к навигации](#Навигация)

In [3]:
data = pd.read_csv(DATA_PATH, index_col=0)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
X = data['text']
y = data['toxic']

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
def lemmatize_text(text):
    return ' '.join([i.lemma_ for i in nlp(text)])


def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).split())

In [8]:
%%time
X = X.apply(lambda x: lemmatize_text(clear_text(x)))

CPU times: total: 42min 51s
Wall time: 45min 6s


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=RANDOM_STATE)

## Обучение

[к навигации](#Навигация)

In [10]:
stopwords = set(nltk_stopwords.words('english'))

### LogReg

[к навигации](#Навигация)

In [11]:
logreg_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 1), stop_words=stopwords)),
    ('tfidf', TfidfTransformer()),
    ('logreg', LogisticRegression(random_state=RANDOM_STATE)),
])

In [12]:
%%time
logreg_gs = GridSearchCV(
    estimator=logreg_pipe,
    param_grid={
        'logreg__max_iter': [80, 100, 120],
        'logreg__C': [.5, 1., 2.],
    },
    scoring='f1',
    n_jobs=-1,
    cv=3,
)
logreg_gs.fit(X_train, y_train)

CPU times: total: 15.9 s
Wall time: 1min 49s


E:\Projects\YandexPracticum\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
                                                                    "aren't",
                                                                    'as', 'at',
                                                                    'be',
                                                                    'because',
                                                                    'been',
                                                                    'before',
                                                                    'being',
                                                                    'below',
                                                                    'between',
                                                                    'both',
                                                                    'but', 'by',
                                                                    'can',
                                                                    'couldn',
                                                                    "couldn't", ...})),
                                       ('tfidf', TfidfTransformer()),
                                       ('logreg',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid={'logreg__C': [0.5, 1.0, 2.0],
                         'logreg__max_iter': [80, 100, 120]},
             scoring='f1')

In [13]:
logreg_model = logreg_gs.best_estimator_
print('Лучшие гиперпараметры:')
for param, value in logreg_gs.best_params_.items():
    print(f'- {param}: {value}')
print(logreg_gs.best_score_)

Лучшие гиперпараметры:
- logreg__C: 2.0
- logreg__max_iter: 80
0.7493171562316673


### CatBoost

[к навигации](#Навигация)

In [14]:
catboost_pipe = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords)),
    ('tfidf', TfidfTransformer()),
    ('catboost', catboost.CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)),
])

In [15]:
%%time
catboost_gs = GridSearchCV(
    estimator=catboost_pipe,
    param_grid={
        'catboost__max_depth': [5, 6],
        'catboost__iterations': [100, 150, 200],
    },
    scoring='f1',
    n_jobs=1,
    cv=3,
)
catboost_gs.fit(X_train, y_train)

CPU times: total: 2h 27min 41s
Wall time: 51min 1s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
                                                                    "aren't",
                                                                    'as', 'at',
                                                                    'be',
                                                                    'because',
                                                                    'been',
                                                                    'before',
                                                                    'being',
                                                                    'below',
                                                                    'between',
                                                                    'both',
                                                                    'but', 'by',
                                                                    'can',
                                                                    'couldn',
                                                                    "couldn't", ...})),
                                       ('tfidf', TfidfTransformer()),
                                       ('catboost',
                                        <catboost.core.CatBoostClassifier object at 0x000002461F9A5FC0>)]),
             n_jobs=1,
             param_grid={'catboost__iterations': [100, 150, 200],
                         'catboost__max_depth': [5, 6]},
             scoring='f1')

In [16]:
print('Лучшие гиперпараметры:')
for param, value in catboost_gs.best_params_.items():
    print(f'* {param}: {value}')
print(catboost_gs.best_score_)

Лучшие гиперпараметры:
- catboost__iterations: 150
- catboost__max_depth: 6
0.7476600142691318


In [17]:
f1_score(y_test, logreg_model.predict(X_test))

0.7596250901225667

## Общие выводы

[к навигации](#Навигация)